<a href="https://www.nvidia.com/dli"> <img src="images/DLI_Header.png" alt="Header" style="width: 400px;"/> </a>

# Speed Up DataFrame Operations w/ RAPIDS cuDF

## Welcome
A **DataFrame** is a 2-dimensional data structure used to represent data in a tabular format, like a spreadsheet or SQL table. Originally offered through the Python Data Analysis ([pandas](https://pandas.pydata.org/docs/)) library, DataFrames have become very popular for its familiar representation along with a robust set of features that are intuitive and expressive. 

Raw data often needs to be manipulated before it can be used for further purposes such as generating **Business Intelligence**, creating **Dashboard Visualization**, or training **Machine Learning** models. These preprocessing steps can include **filtering**, **merging**, **grouping**, and **aggregating**. 

Below is a typical data processing pipeline: 
<p><img src='https://github.com/NVDLI/notebooks/blob/kl/cudf_speed_up/images/flow.png?raw=true' atl='flow' width=1080></p>

According to [studies](https://www.forbes.com/sites/gilpress/2016/03/23/data-preparation-most-time-consuming-least-enjoyable-data-science-task-survey-says/?sh=29f71b266f63), data preparation accounts for ~80% of the work for analysts. This could be due in part to the rapid increase in the size of data as well as the iterative nature of analytics. 

Recognizing this potential bottleneck, NVIDIA created [**cuDF**](https://docs.rapids.ai/api/cudf/stable/) that leverages GPU hardware and software to perform data manipulation tasks with parallel computing, **saving valuable time and resources**. The cuDF library is part of the larger [**RAPIDS**](https://rapids.ai/) data science framework that allows for the execution of **end-to-end analytics pipelines** entirely on GPUs. One of the focus for cuDF and its companion suite of open source software libraries is to provide syntax that is similar to their CPU counterparts, **making it easy to implement**. 

This notebook is intended to demonstrate speedup in data processing by moving common DataFrame operations to the GPU with minimal changes to existing code. 

### Environment Sanity Check
Check the output of `!nvidia-smi` to make sure you've been allocated a RAPIDS supported GPU such as Tesla T4, P4, or P100.

In [1]:
!nvidia-smi

Mon Dec 18 21:31:36 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            On   | 00000000:00:1E.0 Off |                    0 |
| N/A   25C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Interactive Exercise

In [2]:
import numpy as np # for generating sample data

import pandas as df
# import cudf as df
import time # for clocking process times
import matplotlib.pyplot as plt # for visualizing results

class Timer: # creating a Timer helper class to measure execution time
  def __enter__(self):
    self.start=time.perf_counter()
    return self
  def __exit__(self, *args):
    self.end=time.perf_counter()
    self.interval=self.end-self.start

### Loading a Sample Data
We start our demonstration by generating two 2-dimensional arrays of random numbers - we've configured for sizeable arrays at 1MM rows by 50 columns each. Then they are converted to DataFrames using ```pandas.DataFrame()``` or ```cudf.DataFrame()```:

In [3]:
rows=1000000
columns=50

In [4]:
def load_data(): 
  data_a=np.random.randint(0, 100, (rows, columns))
  data_b=np.random.randint(0, 100, (rows, columns))
  dataframe_a=df.DataFrame(data_a, columns=[f'a_{i}' for i in range(columns)])
  dataframe_b=df.DataFrame(data_b, columns=[f'b_{i}' for i in range(columns)])
  return dataframe_a, dataframe_b

with Timer() as process_time: 
  dataframe_a, dataframe_b=load_data()

print(f'The loading process took {process_time.interval:.2f} seconds')
display(dataframe_a.tail(5))
display(dataframe_b.tail(5))

The loading process took 0.96 seconds


,a_0,a_1,a_2,a_3,a_4,a_5,a_6,a_7,a_8,a_9,...,a_40,a_41,a_42,a_43,a_44,a_45,a_46,a_47,a_48,a_49
999995,82,68,9,99,61,55,33,49,33,74,...,91,84,81,88,13,38,98,19,97,86
999996,95,88,59,65,94,54,29,73,71,81,...,62,99,94,67,51,85,81,35,85,87
999997,10,29,75,97,92,8,85,18,66,98,...,97,94,75,5,92,75,54,95,68,63
999998,8,18,71,31,10,11,53,65,13,86,...,75,24,73,90,57,94,79,7,96,59
999999,67,80,61,89,39,40,21,97,45,63,...,67,85,32,12,43,20,71,3,56,37


,b_0,b_1,b_2,b_3,b_4,b_5,b_6,b_7,b_8,b_9,...,b_40,b_41,b_42,b_43,b_44,b_45,b_46,b_47,b_48,b_49
999995,19,34,89,44,49,72,15,67,74,65,...,1,42,11,32,12,61,94,45,9,42
999996,7,9,64,71,96,98,61,10,0,66,...,96,31,82,92,67,84,75,91,11,24
999997,19,49,17,79,29,49,14,14,28,80,...,20,26,3,16,58,14,22,84,57,80
999998,82,91,73,8,28,26,72,96,3,96,...,65,48,70,82,98,21,9,98,76,53
999999,71,66,52,90,75,12,21,10,23,57,...,32,12,39,94,9,48,41,96,17,61


<p><img src='https://github.com/NVDLI/notebooks/blob/kl/cudf_speed_up/images/check.png?raw=true' width=720 atl='check'></p>

We created two DataFrames, _dataframe_a_ and _dataframe_b_ that are 1000000 rows by 50 columns (col_1, col_2, ... col_48, col_49) each. 

### Merging Data
Sometimes data can come from multiple sources and need to be merged into one with ```DataFrame.merge()```. For example, a typical retail data storage infrastructure may include a customer table and separate transaction and product tables. Merging the data allows the correct details to be included in a single DataFrame to get the insight needed. 

In [5]:
def merge_data(left_df, right_df):
  combined_df=df.merge(left_df, right_df, left_index=True, right_index=True)
  return combined_df

with Timer() as process_time: 
  combined_df=merge_data(dataframe_a, dataframe_b)

print(f'The merging process took {process_time.interval:.2f} seconds')
display(combined_df.head())

The merging process took 1.26 seconds


,a_0,a_1,a_2,a_3,a_4,a_5,a_6,a_7,a_8,a_9,...,b_40,b_41,b_42,b_43,b_44,b_45,b_46,b_47,b_48,b_49
0,89,77,1,30,18,36,61,40,57,96,...,13,73,46,37,79,23,47,49,83,53
1,29,38,52,75,45,3,5,87,58,89,...,25,48,48,52,77,83,69,92,17,14
2,28,44,26,68,30,51,53,42,61,71,...,4,41,1,83,37,88,76,80,33,88
3,28,94,53,43,47,70,91,36,33,98,...,74,52,43,18,83,89,56,22,1,74
4,70,31,33,28,10,83,28,27,40,75,...,80,59,83,25,56,83,17,43,84,19


<p><img src='https://github.com/NVDLI/notebooks/blob/kl/cudf_speed_up/images/check.png?raw=true' width=720 atl='check'></p>

We merged two DataFrames, _dataframe_a_ and _dataframe_b_ on their _index_ into one larger DataFrame that is 1000000 rows by 100 columns (a_0, a_1, ..., b_48, b_49). 

### Summarize
Exploring data begins with **descriptive statistics**, which often involves finding the **central tendency** and **dispersion**. They are a quick way to summarize distributions. Measures of central tendency includes the mean, median, and mode - they are used to describe the center of a set of data values. Measures of dispersion include variance and standard deviation - they are used to describe the degree to which data is distributed around the center. We can quickly perform simple descriptive statistics with the ```DataFrame.describe()``` method. 

In [6]:
def summarize(dataframe):
  summary_df=dataframe.describe()
  return summary_df

with Timer() as process_time: 
  summary_df=summarize(combined_df)

print(f'The summarizing process took {process_time.interval:.2f} seconds')
display(summary_df)

The summarizing process took 4.40 seconds


,a_0,a_1,a_2,a_3,a_4,a_5,a_6,a_7,a_8,a_9,...,b_40,b_41,b_42,b_43,b_44,b_45,b_46,b_47,b_48,b_49
count,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,...,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000
mean,49.504284,49.461819,49.513642,49.445297,49.451649,49.507066,49.523366,49.502470,49.496688,49.461338,...,49.505194,49.500555,49.554150,49.502855,49.494660,49.523282,49.479688,49.490541,49.545639,49.511893
std,28.876180,28.853643,28.866280,28.869665,28.868050,28.882414,28.877334,28.840463,28.863719,28.866971,...,28.862368,28.866125,28.856192,28.858374,28.894336,28.866944,28.874169,28.871390,28.871046,28.867298
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,25.000000,24.000000,25.000000,24.000000,24.000000,24.000000,25.000000,25.000000,25.000000,24.000000,...,25.000000,25.000000,25.000000,24.000000,24.000000,25.000000,24.000000,24.000000,25.000000,25.000000
50%,49.000000,49.000000,49.000000,49.000000,49.000000,50.000000,50.000000,50.000000,49.000000,49.000000,...,50.000000,49.000000,50.000000,50.000000,49.000000,50.000000,49.000000,50.000000,50.000000,50.000000
75%,75.000000,74.000000,75.000000,74.000000,74.000000,75.000000,75.000000,74.000000,75.000000,74.000000,...,74.000000,74.000000,75.000000,74.000000,75.000000,75.000000,74.000000,74.000000,75.000000,75.000000
max,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,...,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000


<p><img src='https://github.com/NVDLI/notebooks/blob/kl/cudf_speed_up/images/check.png?raw=true' width=720 atl='check'></p>

Since this is a sample data set, we see that each of columns/features (a_0, a_1, ..., b_48, b_49) have 1000000 values with an average ~50 and standard deviation of ~30

### Correlation - Exploring Relationships
We might be interested in finding relationships/dependencies between two or more variables through their correlation with ```DataFrame.corr()```. Correlation is a number between -1 and 1 that describes the strength of the association between two variables. Two variables with a correlation of 1 suggests that they change together in the same direction while a correlation of -1 suggests that they change together in the opposite direction. 

In [7]:
def correlation(dataframe): 
  corr_df=dataframe.corr()
  return corr_df

with Timer() as process_time: 
  corr_df=correlation(combined_df)

print(f'The correlation process took {process_time.interval:.2f} seconds')
display(corr_df.head())

The correlation process took 23.09 seconds


,a_0,a_1,a_2,a_3,a_4,a_5,a_6,a_7,a_8,a_9,...,b_40,b_41,b_42,b_43,b_44,b_45,b_46,b_47,b_48,b_49
a_0,1.000000,-0.000910,-0.000238,-0.001575,0.001586,0.000601,0.001615,-0.000264,-0.001163,-0.001077,...,0.001119,0.000828,0.000041,-0.001413,-0.000505,-0.000086,-0.000775,0.000856,-0.001493,-0.002113
a_1,-0.000910,1.000000,0.000261,0.001887,-0.000736,-0.000575,-0.001219,-0.000862,-0.000793,0.000262,...,-0.000119,0.000975,0.000896,-0.000288,-0.000626,0.001636,0.000516,0.000197,-0.000127,-0.002004
a_2,-0.000238,0.000261,1.000000,0.000770,0.000641,-0.000058,0.000436,-0.000908,-0.000003,-0.000944,...,0.000159,0.001027,-0.000339,-0.001483,-0.001835,0.001814,0.000369,0.001493,-0.000513,-0.001633
a_3,-0.001575,0.001887,0.000770,1.000000,0.000348,-0.000804,0.002306,0.000228,0.000555,-0.000109,...,0.000975,0.000968,-0.000459,-0.002237,-0.001240,0.000305,0.000197,-0.000936,0.000332,-0.000489
a_4,0.001586,-0.000736,0.000641,0.000348,1.000000,-0.000282,0.000682,-0.001661,0.000413,-0.000470,...,-0.001902,-0.002250,0.003112,-0.002452,-0.000968,-0.000012,-0.000907,0.002554,-0.000348,-0.000291


<p><img src='https://github.com/NVDLI/notebooks/blob/kl/cudf_speed_up/images/check.png?raw=true' width=720 atl='check'></p>

The resulting cross tabulation shows that each column/feature (a_0, a_1, ..., b_48, b_49) have a perfect correlation (1) with itself and is not correlated (~0) with each other. 

### Grouping
We can compare subsets of the data to explore the significance of categories and classes with the ```DataFrame.groupby()``` method. We can even group continuous data values into a smaller number of bins with ```pandas.cut()``` or ```cudf.cut()``` to simplify our analysis. The groupings usually follow an aggregation such as mean or count. For example, we can group our data into 5 equidistant bins based on their sequential index. 

In [8]:
def groupby_summarize(dataframe):
    dataframe['group']=dataframe.index
    dataframe['group']=df.cut(dataframe['group'], 5)
    group_describe_df=dataframe.groupby('group').mean().reset_index(drop=True)
    return group_describe_df

with Timer() as process_time: 
    group_describe_df=groupby_summarize(combined_df)

print(f'The grouping process took {process_time.interval:.2f} seconds')
display(group_describe_df)

The grouping process took 1.06 seconds


,a_0,a_1,a_2,a_3,a_4,a_5,a_6,a_7,a_8,a_9,...,b_40,b_41,b_42,b_43,b_44,b_45,b_46,b_47,b_48,b_49
0,49.546915,49.358660,49.461970,49.434010,49.484750,49.662060,49.524205,49.47993,49.508370,49.438390,...,49.421780,49.461805,49.523175,49.461475,49.498835,49.540690,49.48972,49.488370,49.543250,49.520195
1,49.562080,49.496365,49.448300,49.585025,49.526075,49.459955,49.479580,49.41339,49.484690,49.498110,...,49.473210,49.458885,49.484855,49.612065,49.527255,49.511485,49.54130,49.481570,49.551595,49.499280
2,49.467415,49.489520,49.441445,49.288215,49.435620,49.447975,49.576435,49.50027,49.467480,49.444915,...,49.547135,49.507210,49.553235,49.427325,49.422365,49.452635,49.44793,49.424155,49.509035,49.424910
3,49.469000,49.453680,49.580360,49.426070,49.480340,49.460710,49.598010,49.52355,49.480465,49.472365,...,49.622565,49.623570,49.628595,49.430455,49.491885,49.519795,49.51184,49.532430,49.534890,49.564985
4,49.476010,49.510870,49.636135,49.493165,49.331460,49.504630,49.438600,49.59521,49.542435,49.452910,...,49.461280,49.451305,49.580890,49.582955,49.532960,49.591805,49.40765,49.526180,49.589425,49.550095


<p><img src='https://github.com/NVDLI/notebooks/blob/kl/cudf_speed_up/images/check.png?raw=true' width=720 atl='check'></p>

The resulting DataFrame shows that each group maintains an average of ~50 for each column/feature (a_0, a_1, ..., b_48, b_49) as expected for this sample data. 

### Putting it together
We can measure the total elapsed time for this sample data processing workflow. 

In [ ]:
def pipeline():
    performance={}
    with Timer() as process_time: 
        dataframe_a, dataframe_b=load_data()
    performance['load data']=process_time.interval
    with Timer() as process_time: 
        combined_df=merge_data(dataframe_a, dataframe_b)
    performance['merge data']=process_time.interval
    with Timer() as process_time: 
        summarize(combined_df)
    performance['summarize']=process_time.interval
    with Timer() as process_time: 
        correlation(combined_df)
    performance['correlation']=process_time.interval
    with Timer() as process_time: 
        groupby_summarize(combined_df)
    performance['groupby & summarize']=process_time.interval
    if df.__name__=='cudf': 
        df.DataFrame([performance], index=['gpu']).to_pandas().plot(kind='bar', stacked=True)
    else: 
        df.DataFrame([performance], index=['cpu']).plot(kind='bar', stacked=True)
    return None

### Timing the Pipeline on CPU

In [ ]:
import pandas as df
pipeline()

### Switching to GPU
Traditionally, these tasks are frequently done (as we did) using the popular [**pandas**](https://pandas.pydata.org/) library, which only runs on a single CPU. NVIDIA's [**cuDF**](https://docs.rapids.ai/api/cudf/stable/) library was built with the users in mind - by offering nearly identical syntax to its CPU counterpart, developers only have to make few changes to their existing code to take advantage of its capabilities. 

In [ ]:
import cudf as df

**That's it!** cuDF uses nearly identical syntax to the familiar pandas API. **Brilliant!** It's worth noting that there are some features that are unique to each library, but conviniently there are a lot of overlaps. 

In [ ]:
pipeline()

### Comparing Results
In a trial run, **cuDF** completed the data processing tasks in nearly 10x faster than **pandas**. The expectations is that the speedup will be even more significant as the size of the data becomes largers. Feel free to give it a try by modifying the dimensions of the data above. 

![result](https://github.com/NVDLI/notebooks/blob/kl/cudf_speed_up/images/result.png?raw=true)

## Conclusion
Congratulations on completing the notebook! Want to learn more about cuDF and the rest of the RAPIDS framework? Check out the follow-up to this course, [Accelerating End-to-End Data Science Workflows]('https://courses.nvidia.com/courses/course-v1:DLI+S-DS-01+V1/about') or our other online courses at [NVIDIA DLI]('https://www.nvidia.com/en-us/training/online/').